In [8]:
from pathlib import Path
import json
import re
from tqdm import tqdm
from collections import defaultdict

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
def replace_pattern(text):
    pattern = r'\((.*?)\)\/\((.*?)\)'
    replaced_text = re.sub(pattern, r'\2', text)
    return replaced_text

key_mapper = {
    'hum' : '인문',
    'soc' : '사회',
    'edu' : '교육',
    'eng' : '공학',
    'art' : '예체능',
    'gen' : '교양',
    'lang' : '언어문학',
    'huma' : '인문과학',
    'econ' : '경영경제',
    'soci' : '사회과학',
    'midd' : '중등교육',
    'chil' : '유아교육',
    'spec' : '특수교육',
    'comp' : '컴퓨터통신',
    'arch' : '건축',
    'elec' : '전기전자',
    'desi' : '디자인',
    'pysi' : '무용체육',
    'basi' : '기초',
    'adva' : '학구'
}
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    text_dict = defaultdict(dict)
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            category = key_mapper[line['01_dataset']['5_category']]
            sub_category = key_mapper[line['03_lectureinfo']['3_major_category']]
            lecture_id = line['02_srcinfo']['1_id']
            transcript_id = line['01_dataset']['3_src_path'].split('/')[-1].split('.')[0]
            transcript = replace_pattern(line['06_transcription']['1_text'])
            text_dict[lecture_id]['domain'] = [category, sub_category]
            if 'transcript' not in text_dict[lecture_id]:
                text_dict[lecture_id]['transcript'] = {}
            text_dict[lecture_id]['transcript'][transcript_id] = transcript
            #### data preprocess end 

    for key, value in text_dict.items():
        #### LM(한국어)
        data = {'text': None}
        ## preprocess data from line
        text = '## domain : ' + ' '.join(value['domain']) + '\n'
        transcript = '\n'.join([v for k,v in sorted(value['transcript'].items())])
        
        data['text'] = transcript
        ## preprocess data from line end
        task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
        # print(json.dumps(data, indent=4, ensure_ascii=False))
        #### LM(한국어) end
        # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train:   0%|          | 0/1 [00:00<?, ?it/s]

valid: 100%|██████████| 1/1 [00:01<00:00,  1.58s/it]
